In [1]:
#   x   x   x   x   x   x   x   x   x
#   x   x           x           x   x
#   x       x       x       x       x
#   x           x   x   x           x
#   x   x   x   x   x   x   x   x   x
#   x           x   x   x           x
#   x       x       x       x       x
#   x   x           x           x   x
#   x   x   x   x   x   x   x   x   x

ROWS = 9
COLS = 9

cases = []
for row in range(ROWS):
    rowItems = []
    for col in range(COLS):
        if col == 0: # Première colonne
            rowItems.append('x')
        elif row == 0 or row == ROWS // 2: # Première ligne et ligne du milieu
            rowItems.append('x')
        elif row == ROWS - 1: # Dernière ligne
            rowItems.append('x')
        elif col == COLS - 1 or col == COLS // 2: # Dernière colonne et colonne du milieu
            rowItems.append('x')
        elif col == row or (col + row) + 1 == ROWS: # Diagonales
            rowItems.append('x')
        else:
            rowItems.append(' ') 
    cases.append(rowItems)

for row in range(ROWS):      
    print(cases[row])

['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
['x', 'x', ' ', ' ', 'x', ' ', ' ', 'x', 'x']
['x', ' ', 'x', ' ', 'x', ' ', 'x', ' ', 'x']
['x', ' ', ' ', 'x', 'x', 'x', ' ', ' ', 'x']
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
['x', ' ', ' ', 'x', 'x', 'x', ' ', ' ', 'x']
['x', ' ', 'x', ' ', 'x', ' ', 'x', ' ', 'x']
['x', 'x', ' ', ' ', 'x', ' ', ' ', 'x', 'x']
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']


In [1]:
from model.Case import Case
from model.Joueur import Joueur


joueur = Joueur(nom='aa', prenom='aa')
    
case = Case()
question = case.get_question()
reponse = joueur.reponse_question()

SELECT texte_question, bonne_reponse, choix1, choix2, choix3, choix4 FROM questions ORDER BY RANDOM() LIMIT 1
Quelle est la différence entre les méthodes remove() et pop() pour une liste en Python ?
SELECT texte_question, bonne_reponse, choix1, choix2, choix3, choix4 FROM questions ORDER BY RANDOM() LIMIT 1
Comment supprimer un fichier dans le terminal ?


TypeError: 'NoneType' object is not subscriptable